In [ ]:
---
sidebar_label: AIMind
---

# ChatAIMind

This notebook provides a quick overview for getting started with AIMind (MindsDB) [chat models](/docs/concepts/#chat-models). 

The [Minds Endpoint](https://docs.mdb.ai/) is a unified, OpenAI-compatible API format to interact with various Minds and LLM providers. This standardization simplifies your access to a wide range of functions like completion, embedding, and image generation. It minimizes the learning curve and accelerates your development process.

It provides a wide selection of [advanced large language models (LLMs)](https://docs.mdb.ai/docs/models) from industry-leading AI providers. 

## Setup

To access MindsDB models you'll need to create an MindsDB account, get an API key, and install the `openai` package.

### Credentials

Head to https://mdb.ai/ to sign up to MindsDB and generate an API key. Once you've done this set the MINDSDB_KEY environment variable:

In [11]:
import getpass
import os

os.environ["MINDSDB_API_KEY"] = getpass.getpass("Enter your MINDSDB API key: ")

Enter your MINDSDB API key:  ········


If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [6]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

The LangChain AIMind (MindsDB) integration lives in the `langchain-community` package:

In [ ]:
%pip install langchain-community

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [12]:
from langchain_community.chat_models import ChatAIMind

llm = ChatAIMind(
    model="llama-3-70b",
    temperature=0,
    max_tokens=None,
    # mindsdb_api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # mindsdb_base_url="...",
    # other params...
)

## Invocation

In [14]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Spanish. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Me encanta programar.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 35, 'total_tokens': 42}, 'model_name': 'llama-3-70b', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-447f49b0-7e60-45e4-a448-f546a4786be8-0')

In [15]:
print(ai_msg.content)

Me encanta programar.


## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Italian",
        "input": "I love programming.",
    }
)

AIMessage(content='Ti amo la programmazione.\n\n(Note: "Ti amo" is a more informal way of saying "I love", if you want to use a more formal tone you can say "Io amo la programmazione")', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 30, 'total_tokens': 73}, 'model_name': 'llama-3-70b', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-88b6f59e-d6b6-4799-a661-60c86197acc3-0')